# SCRATCH BOOK

## Verify the coordinates for the rinks in arena_info

In [1]:
# Dependencies
import os
import requests
import pandas as pd

# Path to arena file
arena_file = os.path.join('..','data', 'arena_school_info.csv')
arena_df = pd.read_csv(arena_file)



In [4]:
# import requests
# import pandas as pd

# import requests
# import pandas as pd

# # Define the function to check the location using Google Places API
# def check_location(lat, lng, api_key):
#     url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     params = {
#         'location': f'{lat},{lng}',
#         'radius': 500,  # Distance in meters from the provided coordinates
#         'type': 'stadium',  # Filter search to stadiums/arenas
#         'key': api_key
#     }
    
#     # Debugging: Print the URL and parameters
#     print(f"Requesting URL: {url}")
#     print(f"Parameters: {params}")
    
#     response = requests.get(url, params=params)
    
#     # Debugging: Print the response code and content
#     print(f"Response status code: {response.status_code}")
#     print(f"Response content: {response.text}")
    
#     if response.status_code == 200:
#         results = response.json().get('results')
#         if results:
#             return results[0].get('name'), results[0].get('vicinity')
#         else:
#             return None, "No results found"
#     else:
#         return None, f"API request failed with status {response.status_code}"

# # Define the function to verify coordinates in the DataFrame
# def verify_coordinates(df, api_key):
#     results = []
#     for index, row in df.iterrows():
#         lat = row['Latitude']
#         lng = row['Longitude']
#         arena_name = row['Arena']
        
#         # Debugging: Print the current coordinates and arena being checked
#         print(f"Checking coordinates for arena: {arena_name}")
#         print(f"Latitude: {lat}, Longitude: {lng}")
        
#         # Get the name and vicinity of the nearest stadium/arena
#         name, vicinity = check_location(lat, lng, api_key)
        
#         # Append the original data and verification results
#         results.append({
#             'Arena': arena_name,
#             'Latitude': lat,
#             'Longitude': lng,
#             'Google Places Name': name,
#             'Vicinity': vicinity
#         })
#     return pd.DataFrame(results)

# # Load your API key


# # Assuming arena_df is your DataFrame
# verified_df = verify_coordinates(arena_df, api_key)

# # Output the results
# print(verified_df)


In [5]:
# verified_df.head(10)

In [6]:
## Version 2 of Arena Location verifications
## Returns 5 closest Google Places to coordinates given

import requests
import pandas as pd

# Define the function to check the 5 closest places using Google Places API
def check_nearby_places(lat, lng, api_key):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': f'{lat},{lng}',
        'radius': 500,  # Distance in meters from the provided coordinates
        'key': api_key
    }
    
    # Debugging: Print the URL and parameters being sent to the API
    print(f"Requesting places near lat: {lat}, lng: {lng}")
    print(f"Request URL: {url}")
    print(f"Parameters: {params}")
    
    response = requests.get(url, params=params)
    
    # Debugging: Print the response status and content
    print(f"Response status code: {response.status_code}")
    print(f"Response content: {response.text}\n")  # This shows the full response from the API
    
    if response.status_code == 200:
        results = response.json().get('results')
        if results:
            # Return the top 5 closest places
            return [(result.get('name'), result.get('vicinity')) for result in results[:5]]
        else:
            return [("None", "No results found")]
    else:
        return [("None", f"API request failed with status {response.status_code}")]

# Define the function to verify coordinates and return the 5 closest places
def verify_coordinates(df, api_key):
    results = []
    for index, row in df.iterrows():
        lat = row['Latitude']
        lng = row['Longitude']
        arena_name = row['Arena']
        school_name = row['School']
        
        # Debugging: Print the current arena and coordinates being checked
        print(f"\nChecking nearby places for arena: {arena_name} (School: {school_name})")
        print(f"Latitude: {lat}, Longitude: {lng}")
        
        # Get the 5 closest places
        nearby_places = check_nearby_places(lat, lng, api_key)
        
        # Add each place to the results, along with the original data
        for place in nearby_places:
            results.append({
                'Arena': arena_name,
                'School': school_name,
                'Latitude': lat,
                'Longitude': lng,
                'Google Places Name': place[0],
                'Vicinity': place[1]
            })
            
    return pd.DataFrame(results)

# Load your API key
api_key = ''

# Assuming arena_df is your DataFrame
verified_df = verify_coordinates(arena_df, api_key)

# Output the results
print(verified_df)


NameError: name 'arena_df' is not defined

In [5]:
verified_df.head(10)

## OUTPUT TO TEMP FOLDER FOR MANUAL REVIEW
# output_file = os.path.join('..','TEMP', 'arena_school_info_place_checkV3.csv')
# verified_df.to_csv(output_file, index=False)

